<a href="https://colab.research.google.com/github/jdy5294/Deep_Machine_learning/blob/main/1_4_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_Model_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 기본 라이브러리

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### train_test_split()

In [11]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
X_iris = iris.data
y_iris = iris.target

print(X_iris[:10])
print(y_iris[:10])
print(iris.target_names)
print(iris.feature_names)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]]
[0 0 0 0 0 0 0 0 0 0]
['setosa' 'versicolor' 'virginica']
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [13]:
# 학습 데이터셋으로 예측
dt = DecisionTreeClassifier()

dt.fit(X_iris, y_iris)
pred = dt.predict(X_iris)

print('예측 정확도: {:.4f}'.format(accuracy_score(pred, y_iris)))

예측 정확도: 1.0000


In [19]:
# 데이터 분리 후 예측
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size = 0.3, random_state = 121)

dt = DecisionTreeClassifier()

dt.fit(X_train, y_train)
pred = dt.predict(X_test)

print('예측 정확도: {:.4f}'.format(accuracy_score(pred, y_test)))

예측 정확도: 0.9556


### 교차 검증

#### KFold

In [24]:
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

iris = load_iris()
X_iris = iris.data
y_iris = iris.target

print('붓꽃 데이터 개수: {}'.format(X_iris.shape[0]))

붓꽃 데이터 개수: 150


In [38]:
cv_accuracy = []
kfold = KFold(n_splits = 5)

n_iter = 0

for train_index, test_index in kfold.split(X_iris):
  n_iter += 1
  X_train, X_test = X_iris[train_index], X_iris[test_index]
  y_train, y_test = y_iris[train_index], y_iris[test_index]

  dt = DecisionTreeClassifier(random_state = 121)
  dt.fit(X_train, y_train)
  pred = dt.predict(X_test)
  accuracy = accuracy_score(pred, y_test)
  cv_accuracy.append(accuracy)

  print('{}번째 교차검증 정확도: {:.4f}, train/val 크기 = {}/{}'.format(
      n_iter, accuracy, X_train.shape[0], X_test.shape[0]
  ))

print('\n 평균 검증 정확도: {:.4f}'.format(np.mean(cv_accuracy)))

1번째 교차검증 정확도: 1.0000, train/val 크기 = 120/30
2번째 교차검증 정확도: 1.0000, train/val 크기 = 120/30
3번째 교차검증 정확도: 0.9000, train/val 크기 = 120/30
4번째 교차검증 정확도: 0.9333, train/val 크기 = 120/30
5번째 교차검증 정확도: 0.7333, train/val 크기 = 120/30

 평균 검증 정확도: 0.9133


#### StratifiedKFold

In [39]:
from sklearn.model_selection import StratifiedKFold

cv_accuracy2 = []
stf_kfold = StratifiedKFold(n_splits = 5)

n_iter = 0

for train_index, test_index in stf_kfold.split(X_iris, y_iris):
  n_iter += 1
  X_train, X_test = X_iris[train_index], X_iris[test_index]
  y_train, y_test = y_iris[train_index], y_iris[test_index]

  dt = DecisionTreeClassifier(random_state = 121)
  dt.fit(X_train, y_train)
  pred = dt.predict(X_test)
  accuracy = accuracy_score(pred, y_test)
  cv_accuracy2.append(accuracy)

  print('{}번째 교차검증 정확도: {:.4f}, train/val 크기 = {}/{}'.format(
      n_iter, accuracy, X_train.shape[0], X_test.shape[0]
  ))

print('\n 평균 검증 정확도: {:.4f}'.format(np.mean(cv_accuracy2)))

1번째 교차검증 정확도: 0.9667, train/val 크기 = 120/30
2번째 교차검증 정확도: 0.9667, train/val 크기 = 120/30
3번째 교차검증 정확도: 0.9000, train/val 크기 = 120/30
4번째 교차검증 정확도: 0.9667, train/val 크기 = 120/30
5번째 교차검증 정확도: 1.0000, train/val 크기 = 120/30

 평균 검증 정확도: 0.9600


#### KFold vs StratifiedKFold

    StratifiedKFold는 label의 비율을 고려하여 균일하게 나눔

참고: https://blog.naver.com/PostView.naver?blogId=sjy5448&logNo=222427780700&parentCategoryNo=&categoryNo=144&viewDate=&isShowPopularPosts=true&from=search

In [69]:
# KFold
n_iter = 0
kfold = KFold(n_splits = 3)

for train_index, test_index in kfold.split(X_iris):
  n_iter += 1
  y_train, y_test = y_iris[train_index], y_iris[test_index]
  
  label_train = np.unique(y_train, return_counts=True)
  label_test = np.unique(y_test, return_counts=True)

  count_train = pd.DataFrame(label_train[1], index=label_train[0], columns=['count']).transpose()
  count_test = pd.DataFrame(label_test[1], index=label_test[0], columns=['count']).transpose()
  print('######### Round {} #########'.format(n_iter))
  print('Train 각 label 별 개수 분포')
  print(count_train)
  print('------------------------------')
  print('Validation 각 label 별 개수 분포')
  print(count_test)
  print('------------------------------')

######### Round 1 #########
Train 각 label 별 개수 분포
        1   2
count  50  50
------------------------------
Validation 각 label 별 개수 분포
        0
count  50
------------------------------
######### Round 2 #########
Train 각 label 별 개수 분포
        0   2
count  50  50
------------------------------
Validation 각 label 별 개수 분포
        1
count  50
------------------------------
######### Round 3 #########
Train 각 label 별 개수 분포
        0   1
count  50  50
------------------------------
Validation 각 label 별 개수 분포
        2
count  50
------------------------------


In [70]:
# StratifiedKFold
n_iter = 0
stf_kfold = StratifiedKFold(n_splits = 3)

for train_index, test_index in stf_kfold.split(X_iris, y_iris):
  n_iter += 1
  y_train, y_test = y_iris[train_index], y_iris[test_index]
  
  label_train = np.unique(y_train, return_counts=True)
  label_test = np.unique(y_test, return_counts=True)

  count_train = pd.DataFrame(label_train[1], index=label_train[0], columns=['count']).transpose()
  count_test = pd.DataFrame(label_test[1], index=label_test[0], columns=['count']).transpose()
  print('######### Round {} #########'.format(n_iter))
  print('Train 각 label 별 개수 분포')
  print(count_train)
  print('------------------------------')
  print('Validation 각 label 별 개수 분포')
  print(count_test)
  print('------------------------------')

######### Round 1 #########
Train 각 label 별 개수 분포
        0   1   2
count  33  33  34
------------------------------
Validation 각 label 별 개수 분포
        0   1   2
count  17  17  16
------------------------------
######### Round 2 #########
Train 각 label 별 개수 분포
        0   1   2
count  33  34  33
------------------------------
Validation 각 label 별 개수 분포
        0   1   2
count  17  16  17
------------------------------
######### Round 3 #########
Train 각 label 별 개수 분포
        0   1   2
count  34  33  33
------------------------------
Validation 각 label 별 개수 분포
        0   1   2
count  16  17  17
------------------------------


#### cross_val_score

In [73]:
from sklearn.model_selection import cross_validate, cross_val_score

dt = DecisionTreeClassifier(random_state = 121)

scores = cross_val_score(dt, X_iris, y_iris, scoring = 'accuracy', cv = 5)

print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.9667 0.9667 0.9    0.9667 1.    ]
평균 검증 정확도: 0.96


#### GridSearchCV

In [74]:
from sklearn.model_selection import GridSearchCV

dt = DecisionTreeClassifier()

# 사용 가능한 파라미터 확인
dt.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [83]:
# train_val 분리
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size = 0.2, random_state = 121)

dt_params = {
    'max_depth': [1,2,3,4],
    'min_samples_split':[2,3]
}

# 최적의 파라미터 
dt_grid = GridSearchCV(dt, dt_params, cv = 5)
dt_grid.fit(X_train, y_train)

# 결과 추출
scores_df = pd.DataFrame(dt_grid.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,7
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,7
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,5
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,5
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1
6,"{'max_depth': 4, 'min_samples_split': 2}",0.975000,1
7,"{'max_depth': 4, 'min_samples_split': 3}",0.975000,1


In [84]:
print('GridSearchCV 최적 파라미터:', dt_grid.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(dt_grid.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.9750


In [87]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = dt_grid.best_estimator_

pred = estimator.predict(X_test)
print('예측 정확도: {:.4f}'.format(accuracy_score(pred, y_test)))

예측 정확도: 0.9667
